In [93]:
from bokeh.models.tools import (
    BoxZoomTool, BoxZoomTool, HelpTool, LassoSelectTool, PanTool,
    PreviewSaveTool, ResetTool, ResizeTool, WheelZoomTool,
    TapTool
)

In [95]:
GRAPH_HEIGHT = 1000
GRAPH_WIDTH = 800
TOOLS = [WheelZoomTool(), PanTool(), ResetTool(), TapTool()]

# Import bokeh libraries

In [2]:
from bokeh.plotting import figure, output_file, show
from bokeh import models

Import bokeh, sub modules for making scallter graph with tooltip

In [58]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool

# Import other general libs

In [7]:
import json
import codecs

# load dataset to plot

In [89]:
PATH_TO_DOCUMENT_SVD = './data_for_visual/document_svd_obj.json'
document_svd_result = json.loads(codecs.open(PATH_TO_DOCUMENT_SVD, 'r', 'utf-8').read())

# Make objects to be needed to plot

In [96]:
# This should be function
XY_list = [member_obj['position_vector'] for member_obj in document_svd_result.values()]
universities = [member_obj['university'] for member_obj in document_svd_result.values()]
major = [member_obj['major'] for member_obj in document_svd_result.values()]
height = [member_obj['height'] for member_obj in document_svd_result.values()]
age = [member_obj['age'] for member_obj in document_svd_result.values()]
X = [xy[0] for xy in XY_list]
Y = [xy[1] for xy in XY_list]
labels = [member_obj['member_name'] for member_obj in document_svd_result.values()]
images = [str(member_obj['photo_url'].replace(u'//img', u'/img')) for member_obj in document_svd_result.values()]
blog_links = [member_obj['blog_url'] for member_obj in document_svd_result.values()]

In [97]:
output_file("scatter_tooltip.html")

source = ColumnDataSource(
        data=dict(
        x=X,
        y=Y,
        desc=labels,
        imgs = images,
        univ=universities,
        major=major,
        height=height,
        age=age,
        blog=blog_links
        )
    )

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" height="100" alt="@imgs" width="100"
                    style="float: left; margin: 0px 15px 40px 0px;"
                    border="2"
                >
                <br>
                </img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;"><nobr>@desc さん</nobr></span>
                <span style="font-size: 12px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 13px;">応援ブログ: <A Href="URL" Target="_blank">@univ</A> </span><br>
                <span style="font-size: 10px;">University: @univ</span><br>
                <span style="font-size: 10px;">Major: @major</span><br>
                <span style="font-size: 10px;">age: @age</span><br>
                <span style="font-size: 10px;">height: @height</span><br>
            </div>            
            <div>
                <span style="font-size: 12px;">Position</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """
    )


p = figure(plot_width=GRAPH_WIDTH, plot_height=GRAPH_HEIGHT, tools=[hover]+TOOLS,
           title="Miss Contest Similarity")

p.circle('x', 'y', size=10, source=source)

show(p)